In [55]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [56]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase
import psycopg2
from langchain.prompts import PromptTemplate
import os
from langchain_core.prompts import load_prompt
from langchain_core.output_parsers import StrOutputParser

# 환경 변수 가져오기
# 데이터베이스 접속 정보 설정
username = os.getenv("username")
password = os.getenv("password")
host = os.getenv("host")
port = os.getenv("port")
mydatabase = os.getenv("mydatabase")

# pg_uri 생성
pg_uri = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{mydatabase}"

# SQLDatabase 인스턴스 생성
db = SQLDatabase.from_uri(pg_uri)

In [57]:
print(db.dialect)
# 사용 가능한 테이블 이름들을 출력합니다.
print(db.get_usable_table_names())

postgresql
['apt_buying_tab', 'apt_renting_tab', 'office_buying_tab', 'office_renting_tab']


## 1. 테이블 식별

In [4]:
# model 은 gpt-3.5-turbo 를 지정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [110]:
question = "24년 9월 경기도에서 개인이 매매한 아파트, 오피스텔 건수"
print(question)

24년 9월 경기도에서 개인이 매매한 아파트, 오피스텔 건수


In [111]:
# Load the prompt template
prompt_template = PromptTemplate.from_template(load_prompt("prompts/selectTable.yaml").template)
prompt_template

PromptTemplate(input_variables=['question'], template='#Question : {question}\n\nI\'m trying to create a #Question in SQL. \nPlease provide the table name you need in the #Table Info below.\n\n#Table Info : \n"""\n1. apt_buying_tab 테이블: 아파트 매매 거래 정보\n설명: 이 테이블은 아파트 매매 거래에 관한 상세 정보를 저장합니다. 부동산 시장 분석, 가격 추이 파악, 거래 패턴 분석 등에 활용될 수 있는 중요한 데이터입니다.\n아파트 매매 거래 정보를 담고 있으며, 주택 매매 시장의 동향 파악에 핵심적인 역할을 합니다.\n주요 필드:\n시군구: 아파트가 위치한 행정 구역 정보를 담고 있습니다. 예를 들어, 서울특별시 강남구 등으로 지역별 거래 분석에 활용됩니다.\n단지명: 아파트 단지의 이름으로, 특정 단지의 거래 동향이나 인기 여부를 파악하는 데 사용됩니다.\n전용면적(㎡): 아파트의 전용 면적을 제곱미터로 표시합니다. 전용 면적에 따른 가격 비교나 시장 분석에 중요합니다.\n계약년월: 거래 계약이 체결된 년도와 월을 나타냅니다. 시간에 따른 시장 변화나 시즌별 거래량 분석에 활용됩니다.\n거래금액(만원): 아파트의 실제 거래 금액을 만원 단위로 기록합니다. 가격 추이 분석 및 평가에 사용됩니다.\n매수자: 아파트를 구매한 사람에 대한 정보입니다. 인구 통계학적 분석이나 구매자 특성 파악에 도움을 줍니다.\n매도자: 아파트를 판매한 사람에 대한 정보입니다. 매도자 유형 분석이나 거래 패턴 파악에 사용됩니다.\n건축년도: 아파트가 건축된 연도를 나타냅니다. 건축 연도별 건물의 노후도나 가치 평가에 활용됩니다.\n거래유형: 거래가 이루어진 방식이나 유형을 나타냅니다. 예를 들어, 중개거래, 직접거래 등이 있으며, 거래 형태별 시장 분석에 도움이 됩니다.\n\n2. apt_rent

In [81]:
output_parser = StrOutputParser()

chain = prompt_template | llm | output_parser
answer = chain.invoke({"question" : question})

print(answer)

1. apt_buying_tab
2. office_buying_tab


In [86]:
answer

'1. apt_buying_tab\n2. office_buying_tab'

## 우선, 모델은 GPT-3.5로 고정. 다만, 충분히 바뀔 여지는 있다.

In [58]:
# model 은 gpt-3.5-turbo 를 지정
#llm = ChatOpenAI(model="gpt-4o", temperature=0)
# model 은 gpt-3.5-turbo 를 지정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# LLM 과 DB 를 매개변수로 입력하여 chain 을 생성합니다.
chain = create_sql_query_chain(llm, db)

In [59]:
# chain 을 실행하고 결과를 출력합니다.
generated_sql_query = chain.invoke({"question": " 24년 9월 경기도에서 개인이 매매한 아파트, 오피스텔 건수"})

# 생성된 쿼리를 출력합니다.
print(generated_sql_query.__repr__())

'SELECT COUNT(*) \nFROM apt_buying_tab \nWHERE "계약년월" = 202409 AND "시군구" LIKE \'경기도%\' AND "매수자" = \'개인\'\n\nUNION\n\nSELECT COUNT(*) \nFROM office_buying_tab \nWHERE "계약년월" = 202409 AND "시군구" LIKE \'경기도%\' AND "매수" = \'개인\'\nLIMIT 5;'


In [8]:
type(generated_sql_query)

str

In [9]:
chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], partial_variables={'top_k': '5'}, template='You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tab

- chain 을 실행하면 DB 기반으로 쿼리를 생성합니다.

# AS-IS

In [11]:
# chain 을 실행하고 결과를 출력합니다.
generated_sql_query = chain.invoke({"question": "24년 9월 경기도에서 개인이 매매한 아파트, 오피스텔 건수"})

# 생성된 쿼리를 출력합니다.
print(generated_sql_query.__repr__())
query = generated_sql_query.__repr__()

'SELECT COUNT(*) \nFROM apt_buying_tab \nWHERE "계약년월" = 202409 AND "시군구" LIKE \'경기도%\' AND "매수자" = \'개인\'\n\nUNION ALL\n\nSELECT COUNT(*) \nFROM office_buying_tab \nWHERE "계약년월" = 202409 AND "시군구" LIKE \'경기도%\' AND "매수" = \'개인\'\nLIMIT 5;\n```'


In [12]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# 생성한 쿼리를 실행하기 위한 도구를 생성합니다.
execute_query = QuerySQLDataBaseTool(db=db)

execute_query.invoke({"query": generated_sql_query})

'Error: (psycopg2.errors.SyntaxError) syntax error at or near "```"\nLINE 11: ```\n         ^\n\n[SQL: SELECT COUNT(*) \nFROM apt_buying_tab \nWHERE "계약년월" = 202409 AND "시군구" LIKE \'경기도%%\' AND "매수자" = \'개인\'\n\nUNION ALL\n\nSELECT COUNT(*) \nFROM office_buying_tab \nWHERE "계약년월" = 202409 AND "시군구" LIKE \'경기도%%\' AND "매수" = \'개인\'\nLIMIT 5;\n```]\n(Background on this error at: https://sqlalche.me/e/20/f405)'

### Error 원인

에러 메시지에 따르면 SQL 문에서 "```" 근처에서 구문 오류(syntax error)가 발생했습니다. 이는 SQL 문에 삼중 백틱(```)이 포함되어 있어서 생긴 문제입니다. 삼중 백틱은 마크다운에서 코드 블록을 표시하기 위한 기호로, SQL 문법에서는 사용되지 않습니다.

### TO-BE

### 1. 우선 쿼리 생성

In [13]:
# chain 을 실행하고 결과를 출력합니다.
generated_sql_query = chain.invoke({"question": "24년 9월 경기도에서 개인이 매매한 아파트, 오피스텔 건수"})

# 생성된 쿼리를 출력합니다.
print(generated_sql_query.__repr__())
query = generated_sql_query.__repr__()

'SELECT COUNT(*) \nFROM apt_buying_tab \nWHERE "계약년월" = 202409 AND "시군구" LIKE \'경기도%\' AND "매수자" = \'개인\'\n\nUNION ALL\n\nSELECT COUNT(*) \nFROM office_buying_tab \nWHERE "계약년월" = 202409 AND "시군구" LIKE \'경기도%\' AND "매수" = \'개인\'\nLIMIT 5;\n```'


In [16]:
print(type(query))
print("-"*50)
print(query)

<class 'str'>
--------------------------------------------------
'SELECT COUNT(*) \nFROM apt_buying_tab \nWHERE "계약년월" = 202409 AND "시군구" LIKE \'경기도%\' AND "매수자" = \'개인\'\n\nUNION ALL\n\nSELECT COUNT(*) \nFROM office_buying_tab \nWHERE "계약년월" = 202409 AND "시군구" LIKE \'경기도%\' AND "매수" = \'개인\'\nLIMIT 5;\n```'


### 2. 쿼리 검증 로직 별도로 추가.

In [33]:
llm_4 = ChatOpenAI(model="gpt-4o", temperature=0)
output_parser = StrOutputParser()

print(llm_4)

client=<openai.resources.chat.completions.Completions object at 0x10a9a73d0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10ac2dd10> root_client=<openai.OpenAI object at 0x10ac0e2d0> root_async_client=<openai.AsyncOpenAI object at 0x10ac1f590> model_name='gpt-4o' temperature=0.0 openai_api_key=SecretStr('**********') openai_proxy=''


In [44]:
# Load the prompt template
prompt_template = PromptTemplate.from_template(load_prompt("prompts/edit_prompt.yaml").template)


In [45]:
chain = prompt_template | llm_4 | output_parser
chain

PromptTemplate(input_variables=['query'], template='You are a PostgreSQL expert.\nTake a look at the query below and make sure it is a viable query. If there are errors such as syntax errors, please correct them and provide a valid query.\n\n{query}\n\n# REQUEST : \n# 1. Only provide SQL executable queries out of the box.\n# 2. 벡틱이나 이런 문자 없이 수정된 SQL만 출력할 것.\n# 3. "```" 같은 문자들은 쿼리 수행시킬 때 에러 발생할 수 있기 때문에 제외할 것.\n# 4. 생성된 sql를 통해서 바로 수행해야하기 때문에 sql만 출력할 것.\n\n# Answer Query : \n')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10a9a73d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10ac2dd10>, root_client=<openai.OpenAI object at 0x10ac0e2d0>, root_async_client=<openai.AsyncOpenAI object at 0x10ac1f590>, model_name='gpt-4o', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [46]:
input = {"query": query}
answer = chain.invoke(input)

In [47]:
answer

'SELECT COUNT(*)\nFROM apt_buying_tab\nWHERE "계약년월" = 202409 AND "시군구" LIKE \'경기도%\' AND "매수자" = \'개인\'\n\nUNION ALL\n\nSELECT COUNT(*)\nFROM office_buying_tab\nWHERE "계약년월" = 202409 AND "시군구" LIKE \'경기도%\' AND "매수" = \'개인\'\nLIMIT 5;'

In [48]:
execute_query = QuerySQLDataBaseTool(db=db)

In [51]:
answer_data = execute_query(answer)

In [52]:
answer_data

'[(1687,), (130,)]'

## 답변을 LLM 으로 증강-생성
이전 단계에서 생성한 chain 을 사용하면 답변이 단답형 형식으로 출력됩니다. 이를 LCEL 문법의 체인으로 좀 더 자연스러운 답변을 받을 수 있도록 조정할 수 있습니다.

In [50]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

In [53]:
chain = answer_prompt | llm_4 | output_parser

input = {"question" : "24년 9월 경기도에서 개인이 매매한 아파트, 오피스텔 건수", "query": query, "result" : answer_data}
chat_answer = chain.invoke(input)

In [54]:
chat_answer

'24년 9월 경기도에서 개인이 매매한 아파트 건수는 1687건, 오피스텔 건수는 130건입니다.'